In [2]:
# %connect_info
%qtconsole

In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from graphgym.custom_graphgym.loader.protein import ProteinDataset
# import wei
from 
import matplotlib.pyplot as plt
import os

In [38]:
os.chdir('/Users/cgu3/Documents/Grape-Pi')

Import data from csv fies

In [39]:

# dataset = ProteinDataset("data/yeast-ORBI", numeric_columns=['protein probability'], label_column=None)
# dataset = ProteinBatchDataset("data/yeast-LCQ", numeric_params=['protein probability'])
# dataset = ProteinBatchDataset("data/yeast-ORBI-mRNA", numeric_params=['protein probability', 'mRNA(M)'])
# dataset = ProteinBatchDataset("data/yeast-LCQ-mRNA", numeric_params=['protein probability', 'mRNA(M)'])
# protein_dataset = ProteinDataset("data/single", numeric_columns=['protein_probability'], label_column=None)
# protein_dataset = ProteinDataset("data/single", numeric_columns=['protein_probability', 'mRNA_TPM', 'degree'], label_column='hard_label', rebuild=True)
protein_dataset = ProteinDataset("data/single", numeric_columns=['protein_probability', 'mRNA_TPM'], label_column='hard_label', rebuild=True)

Rebuilding...
Done!


In [55]:
protein_file = [f for f in os.listdir('data/single/raw/protein') if f.endswith('.csv')][0]
df = pd.read_csv(os.path.join('data/single/raw/protein', protein_file))
ids = df['protein.Accession'].values
# df = pd.read_csv('/Users/cgu3/Documents/data/SG_combined_protein_0982_with_protein_probability.csv')
df

,Unnamed: 0,protein.Accession,gene_symbol,pred_prob,protein_probability,soft_label,hard_label,mRNA_TPM,degree
0,0,O75674,TOM1L1,0.836400,0.824788,0.756909,1,2.236920,2.537881
1,1,P60372,KRTAP10-4,0.050054,0.000000,0.000000,0,0.000000,2.430905
2,3,Q15760,GPR19,0.053601,0.000000,0.237121,0,0.166319,0.000000
3,6,Q9H7B7,PKD1L1-AS1,0.048526,0.000000,0.000000,0,0.000000,0.000000
4,7,P61163,ACTR1A,0.934872,0.999955,0.999996,1,3.073551,3.091456
...,...,...,...,...,...,...,...,...,...
12532,20417,A0A3B3IS91,POLGARF,0.048526,0.000000,0.233729,0,0.000000,0.000000
12533,20419,Q495D7,LINC01559,0.343795,0.000000,0.000000,0,4.215585,0.000000
12534,20422,P68133,ACTA1,0.562585,1.000000,1.000000,1,1.636662,3.152965
12535,20424,Q5BVD1,TTMP,0.049242,0.000000,0.243933,0,0.000000,1.841958


In [41]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels, ids, train_mask=None, val_mask=None, test_mask=None):
        self.data = data
        self.labels = labels
        self.ids = ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample_data = self.data[index]
        sample_label = self.labels[index]
        sample_id = self.ids[index]
        return sample_data, sample_label, sample_id


In [42]:
from torch.utils.data import Dataset, Subset
dataset = CustomDataset(protein_dataset.x, protein_dataset.y, ids)

train_indices = torch.nonzero(protein_dataset.train_mask).squeeze().tolist()
train_dataset = Subset(dataset, train_indices)

val_indices = torch.nonzero(protein_dataset.val_mask).squeeze().tolist()
val_dataset = Subset(dataset, val_indices)

test_indices = torch.nonzero(protein_dataset.test_mask).squeeze().tolist()
test_dataset = Subset(dataset, test_indices)


In [43]:
train_dataset[3]

(tensor([0., 0.]), tensor(0.), 'Q9H7B7')

In [44]:
from torch.utils.data import DataLoader
torch.manual_seed(12345)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [45]:
for batch in train_dataloader:
    x, y, id = batch
    print(x.shape)
    print(y.shape)
    print(id)
    break

torch.Size([64, 2])
torch.Size([64])
('Q8NH69', 'O95567', 'P00451', 'O15294', 'A8MUX0', 'O60506', 'O60397', 'P47710', 'Q8N0Z2', 'P29965', 'Q9BZD4', 'Q6NUJ2', 'Q6P2D0', 'Q712K3', 'Q6JBY9', 'O15520', 'Q96A83', 'P68133', 'P51991', 'Q9BYZ6', 'Q9P0K8', 'A0A096LNW5', 'Q5T1J5', 'O15399', 'Q96MZ4', 'P08237', 'P61088', 'Q9P2X8', 'Q969I3', 'P78549', 'O94903', 'Q6UXK5', 'Q9HBY8', 'Q4QY38', 'Q14554', 'Q9UP79', 'Q96N76', 'Q00059', 'Q3KQV3', 'Q9UKJ8', 'Q7KZ85', 'Q9UH77', 'Q8TCD5', 'A0A1B0GTB2', 'Q8NGF7', 'Q8WVK2', 'A0A075B6S6', 'Q6IF00', 'Q9BUW7', 'Q8NHX4', 'Q96T76', 'Q0VG99', 'Q8TD17', 'Q8NHC8', 'Q16690', 'A0A075B6H9', 'A8MYA2', 'P05062', 'Q9NQQ7', 'Q96MY7', 'P18283', 'Q15722', 'Q8N7U7', 'O43345')


In [46]:
import os
from torch import optim, nn, utils, Tensor
import pytorch_lightning as pl
from torchmetrics.classification import BinaryAUROC, BinaryAccuracy, BinaryF1Score
import torch


# define the LightningModule
class MLP(pl.LightningModule):
    def __init__(self, num_features, hidden_channels, num_classes, criterion):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = nn.Linear(num_features, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, num_classes)
        # self.lin3 = nn.Linear(hidden_channels, num_classes)
        self.criterion = criterion
        self.val_auroc = BinaryAUROC()
        self.test_auroc = BinaryAUROC()
        self.val_accuracy = BinaryAccuracy()
        self.test_accuracy = BinaryAccuracy()
        self.val_F1 = BinaryF1Score()
        self.test_F1 = BinaryF1Score()

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        return x

    def get_auc(self, out, target):
        return self.auroc(out, target)

    def training_step(self, batch, batch_idx):
        x, y, id = batch
        logits = self(x).squeeze(-1)  # Perform a single forward pass.

        loss = self.criterion(logits, y)  # Compute the loss solely based on the training nodes.
        values = {"loss": loss}
        self.log_dict(values, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, ids = batch
        logits = self(x).squeeze(-1)  # Perform a single forward pass.
        self.val_accuracy.update(logits, y)
        loss = self.criterion(logits, y)  # Compute the loss solely based on the training nodes.
        self.val_auroc.update(logits, y)
        self.val_F1.update(logits, y)
        values = {"val_loss": loss, "val_acc": self.val_accuracy, "val_auroc": self.val_auroc, "val_F1": self.val_F1}
        self.log_dict(values, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y, ids = batch
        logits = self(x).squeeze(-1)
        self.test_accuracy.update(logits, y)
        loss = self.criterion(logits, y)
        self.test_auroc.update(logits, y)
        self.test_F1.update(logits, y)
        values = {"test_loss": loss, "test_acc": self.test_accuracy, "test_auroc": self.test_auroc, "test_F1": self.test_F1}
        self.log_dict(values, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def predict_step(self, batch, batch_idx):
        x, y, ids = batch
        ids = [item for sublist in ids for item in sublist]
        logits = self(x).squeeze(-1)
        pred_prob = torch.nn.functional.sigmoid(logits)
        return (ids, x[:, 0], x[:, 1], pred_prob, y)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer



In [57]:
# init the autoencoder
# change the in_dim accordingly with the number of features
MLP_model = MLP(2, 10, 1, criterion = torch.nn.BCEWithLogitsLoss())
# MLP_model = MLP(dataset.num_node_features, 10, 2, criterion = torch.nn.CrossEntropyLoss())

In [58]:
import sys
from pytorch_lightning.callbacks import TQDMProgressBar


class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_test_tqdm(self):
        bar = super().init_test_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar



In [59]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=100, enable_progress_bar=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [60]:
trainer.fit(model=MLP_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


  | Name          | Type              | Params
----------------------------------------------------
0 | lin1          | Linear            | 30    
1 | lin2          | Linear            | 11    
2 | criterion     | BCEWithLogitsLoss | 0     
3 | val_auroc     | BinaryAUROC       | 0     
4 | test_auroc    | BinaryAUROC       | 0     
5 | val_accuracy  | BinaryAccuracy    | 0     
6 | test_accuracy | BinaryAccuracy    | 0     
7 | val_F1        | BinaryF1Score     | 0     
8 | test_F1       | BinaryF1Score     | 0     
----------------------------------------------------
41        Trainable params
0         Non-trainable params
41        Total params
0.000     Total estimated model params size (MB)
/Users/cgu3/anaconda3/envs/grape-pi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `Data

In [61]:
trainer.test(model=MLP_model, dataloaders=test_dataloader)

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_F1            0.7756041288375854
        test_acc            0.8444976210594177
       test_auroc           0.9131938219070435
        test_loss           0.6044179201126099
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/Users/cgu3/anaconda3/envs/grape-pi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[{'test_loss': 0.6044179201126099,
  'test_acc': 0.8444976210594177,
  'test_auroc': 0.9131938219070435,
  'test_F1': 0.7756041288375854}]

In [62]:
out = trainer.predict(MLP_model, dataloaders=test_dataloader)
out

/Users/cgu3/anaconda3/envs/grape-pi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[(['P',
   '5',
   '9',
   '0',
   '9',
   '1',
   'Q',
   '9',
   'N',
   'P',
   '8',
   '7',
   'Q',
   '9',
   'N',
   'X',
   '3',
   '8',
   'Q',
   '4',
   '9',
   '5',
   'Z',
   '4',
   'Q',
   '9',
   'U',
   'B',
   'D',
   '9',
   'P',
   '9',
   '8',
   '1',
   '6',
   '8',
   'P',
   '1',
   '8',
   '8',
   '2',
   '5',
   'P',
   '0',
   '4',
   '9',
   '2',
   '1',
   'Q',
   '9',
   'U',
   'K',
   'R',
   '0',
   'Q',
   '9',
   'Y',
   '2',
   '6',
   '7',
   'P',
   '2',
   '5',
   '8',
   '1',
   '5',
   'P',
   '0',
   'D',
   'S',
   'O',
   '1',
   'Q',
   '8',
   'N',
   'E',
   'B',
   '5',
   'A',
   '8',
   'M',
   'Y',
   'Z',
   '5',
   'P',
   '2',
   '2',
   '8',
   '9',
   '1',
   'Q',
   '8',
   'N',
   'F',
   'J',
   '8',
   'P',
   '4',
   '3',
   '0',
   '3',
   '4',
   'A',
   '8',
   'M',
   'Z',
   'A',
   '4',
   'P',
   '3',
   '6',
   '5',
   '7',
   '8',
   'Q',
   '9',
   'N',
   'P',
   '9',
   '1',
   'Q',
   '8',
   'T',
   'C',
   'W',


In [ ]:
accession, raw_prob, mRNA, pred_prob, soft_label = zip(*out)
accession = [item for sublist in accession for item in sublist]
raw_prob = [item.item() for sublist in raw_prob for item in sublist]
mRNA = [item.item() for sublist in mRNA for item in sublist]
pred_prob = [item.item() for sublist in pred_prob for item in sublist]
soft_label = [item.item() for sublist in soft_label for item in sublist]
pd.DataFrame({'accession': accession, 'raw_prob': raw_prob, 'mRNA': mRNA, 'pred_prob': pred_prob, 'soft_label': soft_label})

In [ ]:
test_dataloader

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

# Running the same MLP model but using degree as additional input